<a href="https://colab.research.google.com/github/mtthsmoore/Machine_Learning_course_UGent_D012554_kaggle/blob/master/Kaggle_eye_blinking_prediction_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import warnings;
warnings.filterwarnings('ignore');
import matplotlib.pyplot as plt;
import numpy as np;
import pandas as pd;
import seaborn as sns;
sns.set_context("notebook", font_scale=1.2);
sns.set_style("whitegrid");

## Normalization of test- and trainingset

In [0]:
#Prepare for modelling
.pop(y)
#Normalization
scaler.fit(trainset)
trainset_scaled = scaler.transform(trainset)
testset_scaled = scaler.transform(testset)

## Model selection
Assumptions: binary classification

Models: logistic regression, SVM, decision tree -> try with standard hyperparameters and select 2 best performing algorithms for hyperparameter tuning.

## Regularization (balance of complexity/degree and good fit/accuracy)

**CROSS-VALIDATION**

Use scikit 10-fold cross-validation to compute estimate of generalization performance of the model:

In [0]:
from sklearn.model_selection import cross_val_predict

cv_predictions = cross_val_predict(model,X,y,cv=5)
print(cv_predictions)

To optimize the degree $d$ we can now use the CV loop:

In [0]:
X = dataset[dataset['folds']==0].copy()
y = X.pop('y')
folds = X.pop('folds')

scores = []
scores.append(0) # no features (d=0)
cv_predictions = cross_val_predict(model, X, y, cv=5) # (d=1)
scores.append(metrics.r2_score(y,cv_predictions)) 
for degree in range(2,18,1):
    X['x1^'+str(degree)] = X['x1']**degree
    X['x1^'+str(degree)] = feature_scaler.fit_transform(X[ ['x1^'+str(degree)]])
    cv_predictions = cross_val_predict(model, X, y, cv=5) # (d=1)
    scores.append(metrics.r2_score(y,cv_predictions)) 

tmp = pd.DataFrame(scores,columns=['cv_score'])
tmp.plot(ylim=(0,1))
plt.show()
print("best CV score: {}".format(max(scores)))
print("optimal degree d: {}".format(np.argmax(scores)))

Should we consider this best CV score a good estimated of the generalization performance of a model with optimal degree $d$? No. The CV predictions were used to make the decision on the optimal degree $d$. Instead we need a second CV loop that is nested in the first loop to estimate the generalization performance while minimizing the possibility of overfitting. This is called **nested-CV** and will be explained further in this course.

**TRAIN SET ACCURACY**

Minimizing the cost function $J(\theta)$ thus means minimizing the magnitude of the errors made on the data set while minimizing also the complexity of the model (i.e. small values for $\theta$, preferably zero). The contribution of the model complexity to the cost function is then controlled by **hyperparameter** $\lambda\geq0$, which is typically optimized using cross-validation.

For regularized logistic regression $\lambda\geq0$ is again a model parameter that controls the balance between model training set accuracy and model complexity and is again set by the user. The updates of the model parameters are computed during the gradient descent iterations.

For **logistic regression**: Complexity = C = 1/$\lambda\$

In [0]:
#Logistic regression without polynomial features
from sklearn.linear_model import LogisticRegression

dataset_clf = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_data/master/notebooks/3_logistic_regression/dataset2D.csv")

X = dataset_clf.copy()
y = X.pop('y')
model_clf = LogisticRegression(C=10000)
model_clf.fit(X,y)

#Compute polynomial features for better fit with maximum complexity (overfitting)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

feature_scaler = MinMaxScaler()

X = dataset_clf.copy()
y = X.pop('y')

clf = LogisticRegression(C=1000000)
polynomial_features = PolynomialFeatures(degree=7)
model_pipeline = Pipeline([("polynomial_features", polynomial_features),
                           ("MinMaxScaler", feature_scaler),
                         ("logistic_regression", clf)])

model_pipeline.fit(X,y)

plt.figure(figsize=(12,12))
plot_decision_boundary(model_pipeline,X,y)
plt.show()

We see that the CV performance is optimal for values $C$ arround 50. In this case the average accuracy on the validation sets is 94.2%. However, we cannot consider this accuracy as a performance evaluation of a model with $C=50$. To estimate the performance on unseen external data we have to use an independent test set. However, as the data set is small, leaving out more data might produce unstable results that differ depending on which data points are selected for the test set. 

To solve this we use cross-validation to create the test set as well. By combining the cross_val_predict() and the GridSearchCV() methods of scikit-learn we can create what is know as a **nested cross-validation** loop to compute the predictions on a test set that contains all the data points in the data set:

In [0]:
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

#TRAIN SET: Gridsearch: uses cross-validation to find best hyperparameter
search_space = np.logspace(-10, 18, 10, base=2)

params = dict(logistic_regression__C=search_space)
grid_search = GridSearchCV(model_pipeline, param_grid=params)
grid_search.fit(X, y)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

for mean_score, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("{:.3f} (+/-{:.3f}) for {}".format(mean_score, std * 2, params))

#TEST SET: nested cross-vaidation
params = dict(logistic_regression__C=search_space)
grid_search = GridSearchCV(model_pipeline, param_grid=params)

cv_predictions = cross_val_predict(grid_search, X, y, method="predict_proba")

#EXTRA: hyperparameter tuning
penalty = ['l1', 'l2']
C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}, {1:0.6, 0:0.4}, {1:0.7, 0:0.3}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid = GridSearchCV(estimator=logistic,
                    param_grid=param_grid,
                    scoring='roc_auc',
                    verbose=1,
                    n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

From these predictions we can now compute an estimate of the generalization performance of our optimal model. Here for instance we compute the AUC:

In [0]:
fpr, tpr, thresholds = metrics.roc_curve(y, cv_predictions[:,1])
print(metrics.auc(fpr, tpr))